In [1]:
import numpy as np

def init_proposal(sampler, params):
    return lambda n: sampler(**params, size=n)

def CSMC(obs, N, conditional_traj, proposals_samplers, proposals_dens, transition_dens, obs_dens, init_dens):
    particles = np.array([[]])
    weights = [[]]
    particles[0] = np.append(proposals_samplers[0](N-1), conditional_traj[0][np.newaxis,:])
    for i in range(N):
        weights[0].append(obs_dens[0](obs[0], particles[0, i]) * init_dens(particles[0, i]) / proposals_dens[0](particles[0, i]))
    T = len(obs)
    ancestors = np.array([[]])
    for t in range(1, T):
        ancestors[t-1] = np.append(np.random.choice(range(1, N+1), size=N-1, p=weights[t-1]), N)
        
        new_particles = []
        for i in range(N-1):
            new_particles.append(proposals_samplers[0](1, particles[0:t-1, ancestors[t-1,i]]))
        new_particles.append(conditional_traj[t])
        particles[t] = np.array(new_particles)
        weights.append([])
        for i in range(N):
            particles[0:t, i] = particles[0:t, ancestors[t-1,i]]
            weights[t].append(obs_dens[t](obs[t], particles[0:t+1, i]) * transition_dens[t-1](particles[0:t, ancestors[t-1,i]]) / proposals_dens[t](particles[0:t, ancestors[t-1,i]]))
        


In [2]:
import os
os.chdir('/Users/corentin/Documents/M2/BIPODS/Project/iPMCMC/ipmcmc')

In [3]:
from scipy.spatial.transform import Rotation as R
%run generate_data.py
%run utils.py

np.random.seed(420)

# Parameters
r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)

# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma, transition_coeffs=alpha, observation_coeffs=beta)

In [60]:
l_observations.shape

(50, 20)

In [24]:
transition_dens = np.array([gaussian_density(mu, start_var)] + [gaussian_density_var(omega, alpha)] * (t_max-1))
proposals_dens = np.copy(transition_dens)
obs_dens = np.array([gaussian_density_var(sigma, beta)] * t_max)

proposals_samplers = np.array([lambda n: np.random.multivariate_normal(mu, start_var, size=n)] + [lambda mu: np.random.multivariate_normal(mu[-1], omega)] * (t_max - 1))

conditional_traj = l_states
N = 32

In [51]:
transition_dens[0]


<function gaussian_density.<locals>.dens_func(x)>

In [28]:
%run csmc.py

particles, weights = CSMC(l_observations, N, conditional_traj, proposals_samplers, proposals_dens, transition_dens, obs_dens)

In [29]:
particles

array([[[-0.19030198,  0.63858924,  1.17375978],
        [-0.19030198,  0.63858924,  1.17375978],
        [-0.19030198,  0.63858924,  1.17375978],
        ...,
        [-0.19030198,  0.63858924,  1.17375978],
        [-0.19030198,  0.63858924,  1.17375978],
        [-0.19030198,  0.63858924,  1.17375978]],

       [[ 1.60046878, -1.74668853, -0.09505616],
        [ 1.60046878, -1.74668853, -0.09505616],
        [ 1.60046878, -1.74668853, -0.09505616],
        ...,
        [ 1.60046878, -1.74668853, -0.09505616],
        [ 1.60046878, -1.74668853, -0.09505616],
        [ 1.60046878, -1.74668853, -0.09505616]],

       [[-0.16923772,  2.73990392, -0.87534585],
        [-0.16923772,  2.73990392, -0.87534585],
        [-0.16923772,  2.73990392, -0.87534585],
        ...,
        [-0.16923772,  2.73990392, -0.87534585],
        [-0.16923772,  2.73990392, -0.87534585],
        [-0.16923772,  2.73990392, -0.87534585]],

       ...,

       [[ 7.45184846,  3.76798762, -1.89708207],
        [ 7

In [62]:
l_observations.shape

(50, 20)

True